In [1]:
import urllib.request
import zipfile
import os
from pathlib import Path
import pandas as pd

In [2]:
url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

In [3]:
def download_and_unzip(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
        return

In [4]:
# Downloading the file
with urllib.request.urlopen(url) as response:
    with open(zip_path, "wb") as out_file:
        out_file.write(response.read())

In [5]:
# Unzipping the file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extracted_path)

In [6]:
# Add .tsv file extension
original_file_path = Path(extracted_path) / "SMSSpamCollection"
os.rename(original_file_path, data_file_path)
print(f"File downloaded and saved as {data_file_path}")

File downloaded and saved as sms_spam_collection/SMSSpamCollection.tsv


In [7]:
df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])
df

,Label,Text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
print(df["Label"].value_counts())

Label
ham     4825
spam     747
Name: count, dtype: int64


In [9]:
def create_balanced_dataset(df):
  # Count the instances of "spam"
  num_spam = df[df["Label"] == "spam"].shape[0]
  # Randomly sample "ham' instances to match the number of 'spam' instances
  ham_subset = df[df["Label"] == "ham"].sample(num_spam, random_state=123)
  # Combine ham "subset" with "spam"
  balanced_df = pd.concat([ham_subset, df[df["Label"] == "spam"]])
  return balanced_df

In [10]:
balanced_df = create_balanced_dataset(df)

In [11]:
print(balanced_df["Label"].value_counts())

Label
ham     747
spam    747
Name: count, dtype: int64


In [12]:
balanced_df[balanced_df['Label'] == 'spam'].head()

,Label,Text
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
5,spam,FreeMsg Hey there darling it's been 3 week's n...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...
11,spam,"SIX chances to win CASH! From 100 to 20,000 po..."


In [13]:
balanced_df[balanced_df['Label'] == 'ham'].head()

,Label,Text
4307,ham,Awww dat is sweet! We can think of something t...
4138,ham,Just got to &lt;#&gt;
4831,ham,"The word ""Checkmate"" in chess comes from the P..."
4461,ham,This is wishing you a great day. Moji told me ...
5440,ham,Thank you. do you generally date the brothas?


In [14]:
balanced_df['Label']= balanced_df['Label'].map({"ham":1, "spam":0})

In [15]:
balanced_df['Label'].value_counts()

Label
1    747
0    747
Name: count, dtype: int64

In [16]:
def random_split(df, train_frac, valid_frac):
    df = df.sample(frac = 1, random_state = 123).reset_index(drop=True)
    train_end = int(len(df) * train_frac)
    valid_end = train_end + int(len(df) * valid_frac)
    
    train_df = df[:train_end]
    valid_df = df[train_end:valid_end]
    
    test_df = df[valid_end:]
    
    return train_df,valid_df,test_df

train_df, valid_df, test_df = random_split(balanced_df, 0.7, 0.1)

In [17]:
train_df.to_csv("train_df.csv", index=None)
valid_df.to_csv("valid_df.csv", index=None)
test_df.to_csv("test_df.csv", index=None)

In [18]:
train_df.head(2)

,Label,Text
0,1,Dude how do you like the buff wind.
1,1,Tessy..pls do me a favor. Pls convey my birthd...
